In [1]:
import yellowcab.cleaning

In [2]:
cab_df = yellowcab.io.read_parquet('01.parquet')
cab_df

,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
0,2020-01-01 00:28:15,2020-01-01 00:33:03,1.0,1.20,1.0,238,239,1.0,6.0,3.0,0.5,1.47,0.0,0.3,11.27,2.5
1,2020-01-01 00:35:39,2020-01-01 00:43:04,1.0,1.20,1.0,239,238,1.0,7.0,3.0,0.5,1.50,0.0,0.3,12.30,2.5
2,2020-01-01 00:47:41,2020-01-01 00:53:52,1.0,0.60,1.0,238,238,1.0,6.0,3.0,0.5,1.00,0.0,0.3,10.80,2.5
3,2020-01-01 00:55:23,2020-01-01 01:00:14,1.0,0.80,1.0,238,151,1.0,5.5,0.5,0.5,1.36,0.0,0.3,8.16,0.0
4,2020-01-01 00:01:58,2020-01-01 00:04:16,1.0,0.00,1.0,193,193,2.0,3.5,0.5,0.5,0.00,0.0,0.3,4.80,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6339562,2020-01-31 23:38:07,2020-01-31 23:52:21,1.0,2.10,1.0,163,246,1.0,11.0,0.5,0.5,2.96,0.0,0.3,17.76,2.5
6339563,2020-01-31 23:00:18,2020-01-31 23:19:18,1.0,2.13,1.0,164,79,1.0,13.0,0.5,0.5,3.36,0.0,0.3,20.16,2.5
6339564,2020-01-31 23:24:22,2020-01-31 23:40:39,1.0,2.55,1.0,79,68,1.0,12.5,0.5,0.5,3.26,0.0,0.3,19.56,2.5
6339565,2020-01-31 23:44:22,2020-01-31 23:54:00,1.0,1.61,1.0,100,142,2.0,8.5,0.5,0.5,0.00,0.0,0.3,12.30,2.5


In [3]:
cab_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6339567 entries, 0 to 6339566
Data columns (total 16 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   tpep_pickup_datetime   datetime64[ns]
 1   tpep_dropoff_datetime  datetime64[ns]
 2   passenger_count        float64       
 3   trip_distance          float64       
 4   RatecodeID             float64       
 5   PULocationID           int64         
 6   DOLocationID           int64         
 7   payment_type           float64       
 8   fare_amount            float64       
 9   extra                  float64       
 10  mta_tax                float64       
 11  tip_amount             float64       
 12  tolls_amount           float64       
 13  improvement_surcharge  float64       
 14  total_amount           float64       
 15  congestion_surcharge   float64       
dtypes: datetime64[ns](2), float64(12), int64(2)
memory usage: 773.9 MB


In [4]:
relevant_columns = ['passenger_count', 'trip_distance', 'RatecodeID',
                    'PULocationID', 'DOLocationID', 'payment_type',
                    'fare_amount', 'extra', 'mta_tax', 'tip_amount',
                    'tolls_amount', 'improvement_surcharge', 'total_amount',
                    'congestion_surcharge']

In [5]:
for c in relevant_columns:
    print("---- %s ---" % c)
    print(cab_df[c].value_counts())

---- passenger_count ---
1.0    4547226
2.0     946423
3.0     250234
5.0     225693
6.0     132154
4.0     123470
0.0     114302
7.0         29
9.0         19
8.0         17
Name: passenger_count, dtype: int64
---- trip_distance ---
0.90     117827
0.80     117234
1.00     116409
0.70     111748
1.10     111463
          ...  
36.23         1
91.47         1
54.34         1
46.25         1
30.45         1
Name: trip_distance, Length: 4498, dtype: int64
---- RatecodeID ---
1.0     6126536
2.0      157733
5.0       36296
3.0       13476
4.0        5130
99.0        350
6.0          46
Name: RatecodeID, dtype: int64
---- PULocationID ---
237    292467
161    281545
236    271999
162    235024
186    228231
        ...  
105         2
245         1
30          1
204         1
172         1
Name: PULocationID, Length: 260, dtype: int64
---- DOLocationID ---
236    288835
237    263281
161    249359
170    196167
162    193808
        ...  
59         29
99         16
2           4
105      

In [6]:
x = yellowcab.cleaning.clean_dataset(cab_df, month=1, verbose=True)

--> 114302 invalid entries found in passenger_count
--> 65343 invalid entries found in trip_distance
--> 17104 invalid entries found in fare_amount
--> 2 invalid entries found in extra
--> 0 invalid entries found in mta_tax
--> 0 invalid entries found in tip_amount
--> 0 invalid entries found in tolls_amount
--> 0 invalid entries found in improvement_surcharge
--> 0 invalid entries found in total_amount
--> 0 invalid entries found in congestion_surcharge
196751 invalid entries have been successfully dropped!
--> 155 earlier date entries found in pickup
--> 50 later date entries found in pickup
205 invalid date entries have been successfully dropped!
--> 32 outlier detected and removed from passenger_count column using zscore
--> 40018 outlier detected and removed from trip_distance column using zscore
--> 1719 outlier detected and removed from fare_amount column using HBOS algorithm
--> 24085 outlier detected and removed from extra column using HBOS algorithm
--> 20558 outlier detected

In [7]:
x

,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,RatecodeID,PULocationID,DOLocationID,payment_type,fare_amount,extra,...,total_amount,congestion_surcharge,trip_duration_minutes,pickup_month,pickup_day,pickup_hour,dropoff_month,dropoff_day,dropoff_hour,weekend
0,2020-01-01 00:28:15,2020-01-01 00:33:03,1.0,1.20,Standard rate,238,239,Credit card,6.0,3.0,...,11.27,2.5,4.800000,1,1,0,1,1,0,False
1,2020-01-01 00:35:39,2020-01-01 00:43:04,1.0,1.20,Standard rate,239,238,Credit card,7.0,3.0,...,12.30,2.5,7.416667,1,1,0,1,1,0,False
2,2020-01-01 00:47:41,2020-01-01 00:53:52,1.0,0.60,Standard rate,238,238,Credit card,6.0,3.0,...,10.80,2.5,6.183333,1,1,0,1,1,0,False
3,2020-01-01 00:55:23,2020-01-01 01:00:14,1.0,0.80,Standard rate,238,151,Credit card,5.5,0.5,...,8.16,0.0,4.850000,1,1,0,1,1,1,False
4,2020-01-01 00:09:44,2020-01-01 00:10:37,1.0,0.03,Standard rate,7,193,Cash,2.5,0.5,...,3.80,0.0,0.883333,1,1,0,1,1,0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5967149,2020-01-31 23:00:01,2020-01-31 23:21:31,1.0,4.11,Standard rate,48,75,Credit card,17.5,0.5,...,25.56,2.5,21.500000,1,31,23,1,31,23,False
5967150,2020-01-31 23:38:07,2020-01-31 23:52:21,1.0,2.10,Standard rate,163,246,Credit card,11.0,0.5,...,17.76,2.5,14.233333,1,31,23,1,31,23,False
5967151,2020-01-31 23:00:18,2020-01-31 23:19:18,1.0,2.13,Standard rate,164,79,Credit card,13.0,0.5,...,20.16,2.5,19.000000,1,31,23,1,31,23,False
5967152,2020-01-31 23:24:22,2020-01-31 23:40:39,1.0,2.55,Standard rate,79,68,Credit card,12.5,0.5,...,19.56,2.5,16.283333,1,31,23,1,31,23,False


In [8]:
# how many faulty data do we still got here?
for c in relevant_columns:
    print("---- %s ---" % c)
    print(x[c].value_counts())

---- passenger_count ---
1.0    4358264
2.0     904972
3.0     240205
5.0     218247
6.0     127643
4.0     117823
Name: passenger_count, dtype: int64
---- trip_distance ---
0.90     112057
0.80     111512
1.00     110650
0.70     106211
1.10     106180
          ...  
19.97        36
19.67        35
19.86        34
19.91        32
20.01        27
Name: trip_distance, Length: 2002, dtype: int64
---- RatecodeID ---
Standard rate         5921806
JFK                     41833
Nassau/Westchester       1910
Negotiated fare          1587
Group ride                 15
Newark                      3
Name: RatecodeID, dtype: int64
---- PULocationID ---
237    283547
161    270055
236    264065
162    225777
186    219111
        ...  
201         2
59          2
30          1
251         1
99          1
Name: PULocationID, Length: 254, dtype: int64
---- DOLocationID ---
236    279829
237    255225
161    239161
170    187214
162    185429
        ...  
172         2
176         1
2           1
8